In the 2020 presidential election, the Democratic candidate, Joe Biden, won the election and flipped 5 states (Michigan, Wisconsin, Arizona, Pennsylvania, and Georgia) won by his incumbent counterpart, Donald Trump, in 2016. So, it could be interesting to view how changes in county-level result contributed to his triumph.

Many major U.S.-based news websites have published county-level presidential election results (e.g., www.nbcnews.com, www.usatoday, etc.). So, we can pull the county-level result data from one of these websites to assemble a dataset for later analyses.

Note that, on these websites, the county-level result data is often organized by state and presented by different webpages. And there's often a pattern existing in the URLs of these webpages.

The following gives a list of sample URLs on www.nbcnews.com for webpages that present county-level results by state:

```
"https://www.nbcnews.com/politics/2020-elections/alabama-president-results"             
"https://www.nbcnews.com/politics/2020-elections/arizona-president-results"             
"https://www.nbcnews.com/politics/2020-elections/arkansas-president-results"               
...
"https://www.nbcnews.com/politics/2020-elections/west-virginia-president-results"       
"https://www.nbcnews.com/politics/2020-elections/wisconsin-president-results"           
"https://www.nbcnews.com/politics/2020-elections/wyoming-president-results"

```

In today's in-class exercise, we're going to use the Web page for Georgia's election results only to demonstrate the standard workflow of dynamic Web scraping.

First, let's run the following code snippet to import all necessary tools:


In [6]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get("https://www.webite-url.com")

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


In [5]:
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys

---

Write code to drive a simulated Chrome browser to open the page with the URL `"https://www.nbcnews.com/politics/2020-elections/georgia-president-results"`

In [7]:
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get("https://www.nbcnews.com/politics/2020-elections/georgia-president-results")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


---    

As you can see, one main element on the opened Web page is an HTML table, which collects county-level election results for different candidates. 

By default, the table only displays the election results for a fixed number of counties, while the rest are hidden unless we click the "SHOW ALL COUNTIES" button that follows the table immediately.

So in order to scrape the county-level data for Georgia, we need to first drive the browser to mimic clicking on that button to render the election results completely.
    
When using the Chrome browser's developer tools function to inspect the page's raw content, you will find the piece of HTML source code that underlies the "SHOW ALL COUNTIES" button is as follows:   
 
```html
<button type="button" data-testid="button-press" class="jsx-1765211304 dib button-press founders-cond lh-none f4 fw6 ttu clear-blue bg-white w-100 pv2">
    <div class="jsx-787784943 df flex-row founders-cond justify-center lh-none f4 fw6 ttu">
        <span class="jsx-787784943 pr1">Show all Counties</span><span class="jsx-787784943">
            <svg width="11" height="7" viewBox="0 0 12 8" fill="none" xmlns="http://www.w3.org/2000/svg" class="jsx-3286215947 chevron">
                <path fill-rule="evenodd" clip-rule="evenodd" d="M12 2.06206L5.95831 8L0 2.14401L1.55988 0.58195L5.95831 4.90485L8.19922 2.70243L10.4401 0.5L12 2.06206Z" class="jsx-3286215947"></path>
            </svg>
         </span>
    </div>
</button>
```



Also note that there's another button labeled with "SHOW ALL GEORGIA EXIT POLLS" on the same page. The HTML source code that portrays that button is as follows:

```html
<button type="button" data-testid="button-press" class="jsx-1765211304 dib button-press founders-cond lh-none f4 fw6 ttu clear-blue bg-white">
    <span class="jsx-4193949648 dib ph6 pv2 tu">
        <span data-testid="exit-polls__toggle-button" class="jsx-4193949648 pr1">show all Georgia exit polls</span>
        <svg width="11" height="7" viewBox="0 0 12 8" fill="none" xmlns="http://www.w3.org/2000/svg" class="jsx-3286215947 chevron">
            <path fill-rule="evenodd" clip-rule="evenodd" d="M12 2.06206L5.95831 8L0 2.14401L1.55988 0.58195L5.95831 4.90485L8.19922 2.70243L10.4401 0.5L12 2.06206Z" class="jsx-3286215947"></path>
        </svg>
    </span>
</button>
```

Note that in the above HTML code pieces, most `class` attributes have spaces in their values (e.g., `class="jsx-1765211304 dib button-press founders-cond lh-none f4 fw6 ttu clear-blue bg-white w-100 pv2"`). 

We know that the space, when used in a CSS selector, means selecting the latter element inside the former element, as the latter element is the child of the former one in the document's hierarchy.

However, the space used in class attributes has a different meaning. It means that the corresponding tag belongs to multiple classes. So spaces are used as demlimiters to separate the names of these classes.

Therefore, to identify a tag of multiple classes, we can compose a CSS selector with any of the classes. Or, if we want to make our selection more specific, we can include multiple classes in a selector. For example, we can specify two classes without a space in between in the selector `'button.jsx-1765211304.pv2'` to differentiate the first `button` tag from the second one. 

Write code to locate the first button tag, name the returned object `button`, and click it to load the resutls for all counties:

In [8]:
# Write your code here

button = wd.find_element_by_css_selector("button.jsx-1765211304.pv2")
button.click()

After the election results get fully loaded, write code to instruct Selenium to hand off the page source to Beautiful Soup and name the resulting BeautifulSoup object `election_results`:

In [9]:
# Write your code here
election_results = BeautifulSoup(wd.page_source, 'html.parser')
 


Inspecting the entire page source reveals that the portion of the HTML code for displaying the result table is as follows: 

```html
<div class="jsx-1858457747 df">
    <div class="jsx-2193855077 ">...</div>
    <div class="jsx-1526806201 scrollable-grid">...</div>
    <div class="jsx-3921339373 dn db-m">...</div>
</div>

```

Among the three child `div` tag, only the first 2 are relevant to this scraping task. 

The first child `div` tag displays the column that shows county names, while the second one displays a sub-table that holds the vote data for the 3 named candidates (i.e., Joe Biden, Donald Trump, and Jo Jorgensen) and write-in ones ( `'Write-ins'` represents all write-in candidates). 

Write code to locate the `div` tag and name the returned `Tag` object `result_table`:

In [10]:
# Write your code here
result_table = election_results.find('div', {'class': 'jsx-1858457747'})


You can use the `prettify()` method to inspect contents embedded by the `Tag` object：

In [11]:
print(result_table.prettify())

<div class="jsx-1858457747 df">
 <div class="jsx-2193855077 ">
  <div class="jsx-562154421 founders-cond tl column-heading">
   <div class="jsx-421939043 dn-m df items-end lh-none h-100">
    100
    <!-- -->
    %
    <br class="jsx-421939043"/>
    expected
    <br class="jsx-421939043"/>
    vote in
   </div>
  </div>
  <div class="cell-list">
   <div class="jsx-2568189844 cell-list-heading dn db-m founders-mono gray-80 f3 ws-nowrap tl">
    County
   </div>
   <ul class="jsx-1628370082">
    <li class="jsx-1505179373 founders-mono f3 justify-start" data-testid="cell-list__item">
     <div class="publico-txt truncate f3 pr2 pr9-m ls-normal">
      Appling
      <span class="db dn-m mt1 founders-mono gray-80 f1 ">
       <span class="jsx-998961378 percent pr1">
        100
        <!-- -->
        %
       </span>
       in
      </span>
     </div>
    </li>
    <li class="jsx-1505179373 founders-mono f3 justify-start" data-testid="cell-list__item">
     <div class="publico-txt trun

---

Taking a close look at the source code, you may find the first child `div` tag, which contains county names, has the following structure:


```html

<div class="jsx-2193855077 ">
    ...
    <div class="cell-list">
        <div class="jsx-2568189844 cell-list-heading dn db-m founders-mono gray-80 f3 ws-nowrap tl">County</div>
        <ul class="jsx-1628370082">
            
            <!-- the 1st list item tag-->
            <li data-testid="cell-list__item" class="jsx-1505179373 founders-mono f3 justify-start">
                <div class="publico-txt truncate f3 pr2 pr9-m ls-normal">Appling
                    <span class="db dn-m mt1 founders-mono gray-80 f1 ">
                        <span class="jsx-998961378 percent pr1">100<!-- -->%</span> in
                    </span>
                </div>
            </li>
            
            <!-- the 2nd list item tag-->
            <li data-testid="cell-list__item" class="jsx-1505179373 founders-mono f3 justify-start">
                <div class="publico-txt truncate f3 pr2 pr9-m ls-normal">Atkinson
                    <span class="db dn-m mt1 founders-mono gray-80 f1 ">
                        <span class="jsx-998961378 percent pr1">100<!-- -->%</span> in
                    </span>
                </div>
            </li>
            
            <!-- the 3rd list item tag-->
            <li data-testid="cell-list__item" class="jsx-1505179373 founders-mono f3 justify-start">
                <div class="publico-txt truncate f3 pr2 pr9-m ls-normal">Bacon
                    <span class="db dn-m mt1 founders-mono gray-80 f1 ">
                        <span class="jsx-998961378 percent pr1">100<!-- -->%</span> in
                    </span>
                </div>
            </li>
            
            
            <!-- and so on-->
            ...
            
        </ul>
    </div>
</div>        


```

Write code to extract all county names and maintain them in a Python list named `name_list`. The expected output is

```python
['Appling', 'Atkinson', 'Bacon', 'Baker', 'Baldwin', 'Banks', 'Barrow', 'Bartow', 'Ben Hill', 'Berrien', 'Bibb', 'Bleckley', 'Brantley', 'Brooks', 'Bryan', 'Bulloch', 'Burke', 'Butts', 'Calhoun', 'Camden', 'Candler', 'Carroll', 'Catoosa', 'Charlton', 'Chatham', 'Chattahoochee', 'Chattooga', 'Cherokee', 'Clarke', 'Clay', 'Clayton', 'Clinch', 'Cobb', 'Coffee', 'Colquitt', 'Columbia', 'Cook', 'Coweta', 'Crawford', 'Crisp', 'Dade', 'Dawson', 'Decatur', 'DeKalb', 'Dodge', 'Dooly', 'Dougherty', 'Douglas', 'Early', 'Echols', 'Effingham', 'Elbert', 'Emanuel', 'Evans', 'Fannin', 'Fayette', 'Floyd', 'Forsyth', 'Franklin', 'Fulton', 'Gilmer', 'Glascock', 'Glynn', 'Gordon', 'Grady', 'Greene', 'Gwinnett', 'Habersham', 'Hall', 'Hancock', 'Haralson', 'Harris', 'Hart', 'Heard', 'Henry', 'Houston', 'Irwin', 'Jackson', 'Jasper', 'Jeff Davis', 'Jefferson', 'Jenkins', 'Johnson', 'Jones', 'Lamar', 'Lanier', 'Laurens', 'Lee', 'Liberty', 'Lincoln', 'Long', 'Lowndes', 'Lumpkin', 'McDuffie', 'McIntosh', 'Macon', 'Madison', 'Marion', 'Meriwether', 'Miller', 'Mitchell', 'Monroe', 'Montgomery', 'Morgan', 'Murray', 'Muscogee', 'Newton', 'Oconee', 'Oglethorpe', 'Paulding', 'Peach', 'Pickens', 'Pierce', 'Pike', 'Polk', 'Pulaski', 'Putnam', 'Quitman', 'Rabun', 'Randolph', 'Richmond', 'Rockdale', 'Schley', 'Screven', 'Seminole', 'Spalding', 'Stephens', 'Stewart', 'Sumter', 'Talbot', 'Taliaferro', 'Tattnall', 'Taylor', 'Telfair', 'Terrell', 'Thomas', 'Tift', 'Toombs', 'Towns', 'Treutlen', 'Troup', 'Turner', 'Twiggs', 'Union', 'Upson', 'Walker', 'Walton', 'Ware', 'Warren', 'Washington', 'Wayne', 'Webster', 'Wheeler', 'White', 'Whitfield', 'Wilcox', 'Wilkes', 'Wilkinson', 'Worth']

```

In [12]:
# Write your code here
# Tips: first select all li tags and then write a for loop to traverse them to extract the text only
country_list = result_table.find_all('li',{'class': 'jsx-1505179373'})
name_list = []
for tag in country_list:
  temp = tag.find('div',{'class': 'publico-txt'})
  if temp:
    name_list.append(temp.contents[0])
print(name_list)
 
    

['Appling', 'Atkinson', 'Bacon', 'Baker', 'Baldwin', 'Banks', 'Barrow', 'Bartow', 'Ben Hill', 'Berrien', 'Bibb', 'Bleckley', 'Brantley', 'Brooks', 'Bryan', 'Bulloch', 'Burke', 'Butts', 'Calhoun', 'Camden', 'Candler', 'Carroll', 'Catoosa', 'Charlton', 'Chatham', 'Chattahoochee', 'Chattooga', 'Cherokee', 'Clarke', 'Clay', 'Clayton', 'Clinch', 'Cobb', 'Coffee', 'Colquitt', 'Columbia', 'Cook', 'Coweta', 'Crawford', 'Crisp', 'Dade', 'Dawson', 'Decatur', 'DeKalb', 'Dodge', 'Dooly', 'Dougherty', 'Douglas', 'Early', 'Echols', 'Effingham', 'Elbert', 'Emanuel', 'Evans', 'Fannin', 'Fayette', 'Floyd', 'Forsyth', 'Franklin', 'Fulton', 'Gilmer', 'Glascock', 'Glynn', 'Gordon', 'Grady', 'Greene', 'Gwinnett', 'Habersham', 'Hall', 'Hancock', 'Haralson', 'Harris', 'Hart', 'Heard', 'Henry', 'Houston', 'Irwin', 'Jackson', 'Jasper', 'Jeff Davis', 'Jefferson', 'Jenkins', 'Johnson', 'Jones', 'Lamar', 'Lanier', 'Laurens', 'Lee', 'Liberty', 'Lincoln', 'Long', 'Lowndes', 'Lumpkin', 'McDuffie', 'McIntosh', 'Macon

---

The second child `div` tag portrays 5 columns 

Among them, the first 4 columns hold the county-level election results for the 3 named candidates and write-in ones, while the last one is a spacing column and contains no data.

It has the following structure:


```html
<div class="jsx-1526806201 scrollable-grid">
    <div class="jsx-1526806201 scroll-container">    
        
        <!-- the div tag that corresponds to the column for Joe Biden -->
        <div data-testid="candidate-column">...</div>
        
        <!-- the div tag that corresponds to the column for Donald Trump -->
        <div data-testid="candidate-column">...</div>
        
        <!-- the div tag that corresponds to the column for Jo Jorgensen-->
        <div data-testid="candidate-column">...</div>
        
        <!-- the div tag that corresponds to the column for write-in candidates -->
        <div data-testid="candidate-column">...</div>    
        
        <!-- the div tag that portrays a spacing column without containing any data -->
        <div data-testid="candidate-column">...</div> 
        
    </div>
</div>
```

---

Each `div` tag  with the `data-testid` attribute `"candidate-column"` (except the 5th one) further nests the following child `div` tags (use the column for Joe Biden as an example):

 

```html
<!-- the div tag that shows the candidate's image and name at the top -->
<div class="jsx-4189516194 candidate-results">
    
    <div class="jsx-4189516194 image">
        <div data-test="headshot" class="jsx-576938928 headshot"></div>
        <div class="jsx-2376665660 founders-cond party-block">dem</div>
    </div>
    
    <div class="jsx-4189516194 candidate">
        
        <div class="jsx-4189516194 row">
            <div class="jsx-4189516194 founders-cond name">
                <span class="jsx-4189516194 dn dib-m mb1">Joe Biden</span>
                <span class="jsx-4189516194 dn-m mb-1">Biden</span>
                ...
            </div>
        </div>
        
        <div class="jsx-4189516194 df flex-column flex-row-m items-baseline">
            <div class="jsx-4189516194 founders-cond percent"><span class="jsx-998961378 percent pr1">49.5%</span></div>
            <div class="jsx-4189516194 founders-mono number">
                            <div class="jsx-3437879980 dib number f2 founders-mono">2,473,633</div>
            </div>
        </div>
    </div>
</div>

<!-- the div tag that wraps the subcolumn Percent and the subcolumn Votes   -->
<div class="jsx-3384420229 column-group">
    
    <div class="jsx-2193855077 ">
        <div class="cell-list">
            <div class="jsx-2568189844 cell-list-heading dn db-m founders-mono gray-80 f3 ws-nowrap tc">
                Percent
            </div>
            ...
        </div>
    </div>
    
    <div class="jsx-2193855077 ">
        <div class="cell-list">
            <div class="jsx-2568189844 cell-list-heading dn db-m founders-mono gray-80 f3 ws-nowrap tc">
                Votes
            </div>
            <ul class="jsx-4268314568">
                <!-- the list item tag that shows row 1 -->
                <li data-testid="cell-list__item" class="jsx-1505179373 founders-mono f3 justify-start">
                    <div class="jsx-421939043 pr5 tr w-100">
                        <div class="jsx-3437879980 dib number f2 founders-mono">1,784</div>
                    </div>
                </li>
                <!-- the list item tag that shows  row 2 -->
                <li data-testid="cell-list__item" class="jsx-1505179373 founders-mono f3 justify-start">
                    <div class="jsx-421939043 pr5 tr w-100">
                        <div class="jsx-3437879980 dib number f2 founders-mono">825</div>
                    </div>
                </li>
                
                <!-- and so on -->
                ...
            </ul>
        </div>
    </div>
    ...
</div>

```


Write code to extract candidate names and maintain them in a Python list named `candidate_list`. The expected output is:

```python
['Joe Biden', 'Donald Trump', 'Jo Jorgensen', 'Write-ins']
```



In [19]:
# Write your code here
# Tips: first select all div tags of the class "candidate-results"
# then write a for loop to traverse them to further locate the child div tag of the class "name" 
# and extract the text that corresponds to the candidate's full name only
candidate_results = result_table.contents[1].find_all('div', {'class': 'candidate-results'})
candidate_list = []

for result in candidate_results:
  temp = result.find_all('div',{'class':'name'})
  if temp:
    candidate_list.append(temp[0].find('span').text)

print(candidate_list)

['Joe Biden', 'Donald Trump', 'Jo Jorgensen', 'Write-ins']



---

Next, write code to extract the number of votes received by each of the 4 candidates in each county, and maintain the output in a Python list named `candidate_list`. The expected output should look like the following:

```python

{
    'Joe Biden': [1784, 825, 625, 652, 9140, 932, 10453, 12091, 2393, 1269, 43408, 1312, 700, 2791, 6738, 11248, 5208, 3274, 1263, 7967, 1269, 16236, 6932, 1105, 78247, 667, 1854, 42779, 36055, 791, 95466, 744, 221847, 4511, 4190, 29232, 2059, 24210, 1615, 2982, 1261, 2486, 4782, 308162, 2172, 1911, 24568, 42814, 2450, 167, 7718, 2879, 2886, 1324, 2570, 33062, 11917, 42208, 1593, 380212, 2932, 155, 15882, 4384, 3619, 4087, 241994, 3562, 25033, 2976, 1791, 5457, 3157, 824, 73443, 32239, 1008, 7642, 1761, 1028, 4058, 1266, 1222, 4882, 2620, 1019, 8074, 4558, 13104, 1432, 2035, 20116, 3126, 4168, 2612, 2858, 3411, 1312, 4287, 748, 3993, 4385, 980, 3353, 2301, 49446, 29789, 8162, 2439, 29695, 5922, 2824, 1100, 1505, 3657, 1230, 3448, 497, 1984, 1671, 59119, 31237, 462, 2661, 1256, 11828, 2386, 1182, 6314, 2114, 561, 2062, 1388, 1488, 2376, 8708, 5318, 2938, 1550, 952, 11577, 1409, 2044, 2800, 4203, 5770, 12683, 4169, 1468, 4743, 2688, 640, 689, 2411, 10680, 861, 2160, 2074, 2395], 
    'Donald Trump': [6570, 2300, 4017, 897, 8903, 7795, 26804, 37672, 4111, 6419, 26559, 4329, 6993, 4261, 14240, 18387, 5400, 8406, 923, 15249, 3133, 37476, 25167, 3419, 53232, 880, 8064, 99585, 14450, 637, 15811, 2105, 165436, 10578, 11777, 50013, 4900, 51501, 4428, 4985, 6066, 13398, 6755, 58377, 5843, 2159, 10441, 25454, 2710, 1256, 23361, 6226, 6553, 2888, 12169, 37956, 28906, 85123, 9069, 137247, 13429, 1402, 25617, 19405, 7034, 7066, 166400, 16637, 64183, 1154, 12330, 14319, 9465, 4519, 48259, 41540, 3134, 29502, 5822, 4695, 3537, 2161, 2850, 9940, 6331, 2509, 14493, 12007, 7959, 3173, 3527, 25692, 12163, 6169, 4016, 1783, 11326, 2275, 6524, 2066, 4935, 11057, 2960, 8231, 12944, 30107, 23869, 16595, 5592, 54517, 6506, 14110, 7898, 9127, 13587, 2815, 8291, 604, 7474, 1390, 26780, 13014, 1800, 3915, 2613, 18104, 9367, 801, 5733, 1392, 360, 6054, 2420, 2825, 2004, 12969, 10784, 7873, 6384, 2101, 18142, 2349, 2370, 12650, 8606, 23173, 37839, 9903, 1166, 4668, 9987, 748, 1583, 12222, 25644, 2402, 2823, 2665, 6830], 
    'Jo Jorgensen': [36, 30, 25, 6, 208, 74, 664, 701, 60, 55, 747, 67, 56, 49, 357, 455, 75, 91, 12, 470, 29, 760, 494, 44, 1929, 35, 132, 2451, 841, 7, 1053, 12, 6445, 125, 119, 1330, 76, 1088, 59, 66, 107, 197, 88, 4207, 56, 35, 278, 838, 28, 18, 492, 66, 66, 35, 110, 976, 518, 1980, 103, 6320, 164, 8, 489, 244, 54, 91, 5629, 232, 1321, 23, 125, 215, 106, 51, 1296, 1059, 26, 531, 61, 48, 44, 28, 28, 112, 94, 48, 164, 149, 331, 36, 95, 547, 242, 118, 68, 22, 200, 38, 66, 20, 34, 148, 27, 122, 144, 961, 576, 411, 102, 1160, 125, 233, 49, 88, 152, 36, 116, 5, 110, 12, 1110, 430, 13, 51, 19, 279, 132, 7, 101, 16, 7, 69, 34, 21, 36, 195, 177, 103, 45, 24, 328, 33, 30, 108, 96, 411, 571, 117, 16, 66, 104, 3, 13, 183, 442, 16, 47, 31, 60], 
    'Write-ins': [0, 0, 0, 0, 10, 4, 25, 22, 0, 1, 49, 2, 0, 0, 14, 19, 2, 5, 0, 12, 0, 30, 33, 0, 76, 0, 3, 44, 75, 1, 61, 1, 294, 0, 1, 45, 2, 46, 0, 3, 13, 3, 3, 131, 1, 1, 3, 33, 0, 1, 8, 5, 2, 1, 0, 18, 24, 66, 2, 152, 2, 0, 6, 11, 0, 3, 327, 11, 65, 0, 2, 11, 6, 0, 18, 34, 0, 10, 3, 3, 0, 0, 0, 6, 0, 0, 1, 5, 0, 0, 3, 20, 14, 1, 0, 0, 8, 1, 3, 0, 1, 2, 1, 4, 6, 14, 29, 25, 5, 45, 1, 5, 1, 1, 3, 0, 2, 0, 6, 0, 68, 18, 0, 2, 0, 8, 0, 0, 2, 2, 0, 2, 0, 1, 0, 5, 1, 0, 0, 0, 10, 0, 0, 4, 2, 20, 5, 0, 0, 3, 3, 0, 0, 15, 0, 0, 1, 0, 0]}
```


In [20]:
# Write your code here
# Tips: first select all div tags of the class "column-group"
# then write a for loop to process the subcolumn Votes for each of the 4 candidates 
# The rowwise text extraction requires a second-level for loop

# You can use the expression int(''.join([c for c in vote_tag.get_text() if c!=',']))
# to remove textual commas and convert the string form of the data to an integer value
candidate_vote = {}

vote_results = result_table.find_all('div',{'class': 'column-group'})
for i, candidate in enumerate(vote_results[:4]):
  temp = candidate.contents[1]
  vote_list_raw = temp.find_all('div',{'class':'jsx-3437879980'})
  vote_list_num = []
  for vote in vote_list_raw:
    vote_list_num.append(int(vote.text.replace(',','')))
  candidate_vote[candidate_list[i]] = vote_list_num

print(candidate_vote)




{'Joe Biden': [1784, 825, 625, 652, 9140, 932, 10453, 12091, 2393, 1269, 43408, 1312, 700, 2791, 6738, 11248, 5208, 3274, 1263, 7967, 1269, 16236, 6932, 1105, 78247, 667, 1854, 42779, 36055, 791, 95466, 744, 221847, 4511, 4190, 29232, 2059, 24210, 1615, 2982, 1261, 2486, 4782, 308162, 2172, 1911, 24568, 42814, 2450, 167, 7718, 2879, 2886, 1324, 2570, 33062, 11917, 42208, 1593, 380212, 2932, 155, 15882, 4384, 3619, 4087, 241994, 3562, 25033, 2976, 1791, 5457, 3157, 824, 73443, 32239, 1008, 7642, 1761, 1028, 4058, 1266, 1222, 4882, 2620, 1019, 8074, 4558, 13104, 1432, 2035, 20116, 3126, 4168, 2612, 2858, 3411, 1312, 4287, 748, 3993, 4385, 980, 3353, 2301, 49446, 29789, 8162, 2439, 29695, 5922, 2824, 1100, 1505, 3657, 1230, 3448, 497, 1984, 1671, 59119, 31237, 462, 2661, 1256, 11828, 2386, 1182, 6314, 2114, 561, 2062, 1388, 1488, 2376, 8708, 5318, 2938, 1550, 952, 11577, 1409, 2044, 2800, 4203, 5770, 12683, 4169, 1468, 4743, 2688, 640, 689, 2411, 10680, 861, 2160, 2074, 2395], 'Donald Tru

After oragnizing the vote data by candidate in the Python dictionary, 
run the following code snippet to pack them into a tabular form implemented by a Pandas DataFrame object:

In [21]:
import pandas as pd

pd.DataFrame(candidate_vote, index=name_list)

,Joe Biden,Donald Trump,Jo Jorgensen,Write-ins
Appling,1784,6570,36,0
Atkinson,825,2300,30,0
Bacon,625,4017,25,0
Baker,652,897,6,0
Baldwin,9140,8903,208,10
...,...,...,...,...
Whitfield,10680,25644,442,0
Wilcox,861,2402,16,0
Wilkes,2160,2823,47,1
Wilkinson,2074,2665,31,0
